In [51]:
import pandas as pd
import numpy as np

In [3]:
# Dataframes from Census-Data
census_column_keys = pd.read_csv("../shared_data_read_only/Census-Data/ACSST5Y2022.S2201-Column-Metadata.csv")
census_data = pd.read_csv("../shared_data_read_only/Census-Data/ACSST5Y2022.S2201-Data.csv", low_memory=False)
nonprofit_data = pd.read_csv("../shared_data_read_only/Census-Data/Nonprofit_Data_PA.csv")

In [4]:
# Dataframes from Education-Data
enrollment_demographics = pd.read_csv("../shared_data_read_only/Education-Data/2023-2024 Enrollment & Demographics.csv")
master_school_list = pd.read_csv("../shared_data_read_only/Education-Data/2023-2024 Master School List (20231003).csv")
college_matriculation = pd.read_csv("../shared_data_read_only/Education-Data/SDP_College_Matriculation_2021-2022.csv")
graduation_rates = pd.read_csv("../shared_data_read_only/Education-Data/SDP_Graduation_Rates_School_S_2023-02-15.csv")
school_metrics = pd.read_csv("../shared_data_read_only/Education-Data/SPREE_SY2122_School_Metric_Scores.csv")
student_ada = pd.read_csv("../shared_data_read_only/Education-Data/Student_ADA_Yearly.csv")

In [17]:
# Dataframes from Geography-Mapping
geography_map = pd.read_csv("../shared_data_read_only/Geography-Mapping/ZIP_TRACT_122023.csv")

In [36]:
#Clean and merge the census data

#check if NAME column contains "Pennsylvania"
#list(census_data.columns)
PA_census_data = census_data[census_data['NAME'].str.contains('Pennsylvania')]
print(PA_census_data.shape[0]) #this has 3446 because of multiple tracts

#remove the leading "1400000US" in front to extract the tract number
PA_census_data.loc[:, 'GEO_ID'] = PA_census_data['GEO_ID'].str.replace('1400000US', '')
PA_census_renamed = PA_census_data.rename({'GEO_ID' : 'TRACT'}, axis=1)

#filter down the geography map
PA_geography_map = geography_map[geography_map['USPS_ZIP_PREF_STATE'].str.contains('PA')]
print(PA_geography_map.shape[0]) #8907 values, greater than our 3446 so not every tract is represented


3446
8907


In [89]:
#convert TRACT numbers to strings
PA_census_renamed['TRACT'] = PA_census_renamed['TRACT'].astype(str)
PA_geography_map.loc[:, 'TRACT'] = PA_geography_map['TRACT'].astype(str)

#map the tract number to the zip code
merged_census = pd.merge(PA_census_renamed, PA_geography_map, on='TRACT', how='left')

#move zip to be 1st column
columns = ['ZIP'] + [col for col in merged_census.columns if col != 'ZIP']

#reassign the column order
merged_census = merged_census[columns] #contains census data and geographical information so far

# sort by zip code so we can see what is happening and how to collapse
merged_census.sort_values(by='ZIP', ascending=True, inplace=True)

print(merged_census.shape[0])
print(merged_census.head(10))

#convert object types to numeric data
object_cols = merged_census.select_dtypes(include=['object']).columns

for col in object_cols:
    merged_census[col] = pd.to_numeric(merged_census[col], errors='coerce')

#get the numeric columns only
numeric_cols = merged_census.select_dtypes(include=[np.float64, np.float32]).columns.tolist()
numeric_cols = [col for col in numeric_cols if col not in ['ZIP', 'Unnamed: 458']]

# make sure zip is not part of the averaging columns
print("Columns for averaging:", numeric_cols)


# collpase by averageing out the values in each column
collapsed_data = merged_census.groupby('ZIP')[numeric_cols].median().reset_index()

print(collapsed_data.shape[0])
print(collapsed_data.head(10))

8908
          ZIP        TRACT                                               NAME  \
1101  15001.0  42007604901  Census Tract 6049.01; Beaver County; Pennsylvania   
1064  15001.0  42007602900     Census Tract 6029; Beaver County; Pennsylvania   
1068  15001.0  42007603000     Census Tract 6030; Beaver County; Pennsylvania   
1073  15001.0  42007603202  Census Tract 6032.02; Beaver County; Pennsylvania   
1128  15001.0  42007605500     Census Tract 6055; Beaver County; Pennsylvania   
1104  15001.0  42007605001  Census Tract 6050.01; Beaver County; Pennsylvania   
1103  15001.0  42007604902  Census Tract 6049.02; Beaver County; Pennsylvania   
1107  15001.0  42007605002  Census Tract 6050.02; Beaver County; Pennsylvania   
1133  15001.0  42007605700     Census Tract 6057; Beaver County; Pennsylvania   
1099  15001.0  42007604800     Census Tract 6048; Beaver County; Pennsylvania   

     S2201_C01_001E S2201_C01_001M S2201_C01_002E S2201_C01_002M  \
1101           1636            141 

In [88]:
print(merged_census.info())

<class 'pandas.core.frame.DataFrame'>
Index: 8908 entries, 1101 to 3864
Columns: 466 entries, ZIP to TOT_RATIO
dtypes: float64(6), object(460)
memory usage: 31.7+ MB
None


In [43]:
#Adding in educational data
#master_school_list_renamed = master_school_list.rename({'Zip Code' : 'ZIP'}, axis=1)
#master_school_list_renamed.head(10)
#master_school_list_renamed.

#merged_school_info = pd.merge(master_school_list, merged_census, on='ZIP'



In [ ]:
#Filter down to just Pennslyvania

In [ ]:
#Filter down the number of parameters in our data (some of these features are not necessarily the most important!)




In [ ]:
#Average the schools within each zip code



In [ ]:
#Filter data down to radius of 15 miles (see list of generate zip codes)